In [1]:
import numpy as np
import time
import copy
import random

In [258]:
import gym
import numpy.ma as ma

In [2]:
nSamples = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
machineNames = ['m0','m1','m2','m3','m4']

In [3]:
# p - persistance
# r - ready time

# Cj oznacza moment zakończenia wykonywania zadania Jj w uszeregowaniu

# p r 
# r  >= c - p

# Game - Generate

In [274]:
class ptszGenerator(gym.Env):
    def __init__(self, k, N):
        self.action_space = k
        self.observation_space = N
        
        self.mashines = None
        self.tasks = None
        self.observations = None
    
    
    def seed(self, seed=None):
        print("seed not implemented")
        pass        
    
    
    def step(self, actions):
        if (actions == self.action_space).all(): #if action is do nothing then increase current time
            self.observations[-1] += 1
            return self.observations
        
        for i,a in enumerate(actions):
            if a == self.action_space: # machine 'a' does nothing
                continue
            self.observation[self.action_space*2 + 3*a] = 1 # mark tasks as done

            current_time = self.observations[-1]
            r,p,_ = self.observation[self.action_space*2 + 3*a: self.action_space*2 + 3*(a+1)] # take r,p,done of selected task
            
            speed = self.observations[2*i]
            self.observations[2*i+1] = current_time + p / speed #ready_time of the current mashine
        
    
    def reset(self):
        # returns: observation: machines, tasks, current_time
        #                       [speed, ready_time]*k, [r, p, done]*N, current_time
        # shape: (1,2*k+3*N+1)
        timeCounter = 0
        tasks = []
        for _ in range(self.observation_space):
            p = np.random.randint(1, 100)
            r = np.random.randint(timeCounter - 5 if timeCounter - 5 > 0 else 0 ,timeCounter + 2)
            timeCounter += int(p/3)
            
            #ready, persistance, doneBoolean
            tasks.append([p,r,0])
        #random.shuffle(tasks) ///////////////////////////////////////////// to uncomment
        self.tasks = np.array(tasks)
        
        m0 = np.random.randint(50,100)
        m1 = np.random.randint(50,100)
        m2 = np.random.randint(50,60)
        m3 = np.random.randint(50,100)
        m4 = np.random.randint(99,100)
        machines = np.array([ [m0,0], [m1,0], [m2,0] ,[m3,0] ,[m4,0]], dtype="float")
        machines[:,0] =  machines[:,0] /max(machines[:,0])

        index = range(5)
        indicies = np.random.choice( index, 5, replace=False )
        self.machines = machines[indicies]

        mechineTasks = np.concatenate((machines,tasks), axis=None)
        self.observations = np.concatenate((mechineTasks, 0), axis=None)
        
        return self.observations
    
        
    def get_valid_tasks(self):
        readyToGo = np.ones(self.observation_space)
        timeCounter = self.observations[-1]
        for i,obs in enumerate(range(self.action_space*2, self.action_space*2+self.observation_space*3, 3)):
            p,r,done = self.observations[[obs,obs+1,obs+2]]
            if r <= timeCounter and done==0: # task is ready and not done
                readyToGo[i] = 0
                                 
        return readyToGo
                               
    
    def get_valid_machines(self):
        readyMachines = []
        timeCounter = self.observations[-1]
        for i in range(self.action_space):
            speed = self.observations[2*i]
            readyTime = self.observations[2*i+1]
            
            if readyTime <= timeCounter:
                readyMachines.append(i)
        return np.array(readyMachines)
    
    
    def _get_obs(self):
        return self.observations
    
    
    def preprocess_action(self, action):
        # input: actions in shape (n,k)
        #output: actions in shape (n), where each number means id of task to take od sleep for number== -1
        filteredAction = []
        mask = np.array((*validT, 0)) # 1 means invalid
        
        for a in action:
            mask_arr = ma.masked_array(a, mask=mask) 
            argmax = mask_arr.argmax()
            
            if argmax == self.observation_space:
                filteredAction.append(argmax)
                continue
            mask[argmax] = 1
            filteredAction.append(argmax)
        return np.array(filteredAction)
        
        
    def print_observations(self):
        for i in range(self.action_space):
            print(f"machine_{i} speed: {round(self.observations[2*i],2)} readyTime: {round(self.observations[2*i+1],2)}")
        for j in range(self.observation_space):
            print(f"task_{j+1} p: {self.observations[2*(i+1)+j*3]} r: {self.observations[2*(i+1)+j*3+1]} done: {self.observations[2*(i+1)+j*3+2]}")
        
        print(self.observations[-1])
    
    
    def sample(self):
        return np.random.rand(5,self.observation_space+1) #+1 because of sleep behaviour
                               

In [275]:
x = ptszGenerator(5,10)
x.reset()


array([  0.90909091,   0.        ,   0.50505051,   0.        ,
         0.54545455,   0.        ,   0.60606061,   0.        ,
         1.        ,   0.        ,  94.        ,   0.        ,
         0.        ,  31.        ,  32.        ,   0.        ,
        91.        ,  36.        ,   0.        ,  83.        ,
        67.        ,   0.        ,  75.        ,  99.        ,
         0.        ,  28.        , 123.        ,   0.        ,
        82.        , 131.        ,   0.        ,  75.        ,
       160.        ,   0.        ,   2.        , 185.        ,
         0.        ,  55.        , 185.        ,   0.        ,
         0.        ])

In [287]:
x.print_observations()

machine_0 speed: 0.91 readyTime: 10.0
machine_1 speed: 0.51 readyTime: 0.0
machine_2 speed: 0.55 readyTime: 0.0
machine_3 speed: 0.61 readyTime: 0.0
machine_4 speed: 1.0 readyTime: 0.0
task_1 p: 94.0 r: 0.0 done: 0.0
task_2 p: 31.0 r: 32.0 done: 0.0
task_3 p: 91.0 r: 36.0 done: 0.0
task_4 p: 83.0 r: 67.0 done: 0.0
task_5 p: 75.0 r: 99.0 done: 0.0
task_6 p: 28.0 r: 123.0 done: 0.0
task_7 p: 82.0 r: 131.0 done: 0.0
task_8 p: 75.0 r: 160.0 done: 0.0
task_9 p: 2.0 r: 185.0 done: 0.0
task_10 p: 55.0 r: 185.0 done: 0.0
1.0


In [288]:
x.get_valid_machines()

array([1, 2, 3, 4])

In [286]:
x.observations[1] = 10

In [270]:
action = x.sample()

In [273]:
filteredAction = []
mask = np.array((*validT, 0))
for a in action:
    mask_arr = ma.masked_array(a, mask=mask) 
    argmax = mask_arr.argmax()
    print(argmax)
    if argmax == 10:
        filteredAction.append(argmax)
        continue
    mask[argmax] = 1
    filteredAction.append(argmax)
print(action)
print(filteredAction)


10
0
10
10
10
[[0.39005464 0.39586013 0.28937759 0.60535998 0.20506988 0.8972126
  0.73610527 0.78599821 0.8801794  0.7909681  0.99264441]
 [0.11000979 0.16664407 0.28882336 0.62996815 0.96573818 0.16051714
  0.34933869 0.88929474 0.64960811 0.40803466 0.00585061]
 [0.16025271 0.29618676 0.73912635 0.28121251 0.56156314 0.44570667
  0.87926453 0.9207871  0.0133838  0.2556306  0.39154247]
 [0.65630056 0.94191498 0.46523368 0.33830402 0.97878254 0.20650305
  0.83442698 0.52748261 0.80156281 0.63570912 0.97021645]
 [0.24372975 0.52132919 0.4931095  0.74957894 0.04225409 0.90198235
  0.90487613 0.80054499 0.50214038 0.19740732 0.00355026]]
[10, 0, 10, 10, 10]


In [244]:
a = action[0]

In [255]:
mask = validT==1
mask = np.array((*mask,True))

In [257]:
a[mask]

array([0.94930191, 0.8845636 ])

In [84]:
x

array([0.52525253, 0.        , 0.8989899 , 0.        , 0.57575758,
       0.        , 0.5959596 , 0.        , 1.        , 0.        ])

56

In [105]:
for x in nSamples:
    timeCounter = 0
    fileData = []
    fileData.append(x)
    for _ in range(x):
        p = np.random.randint(1, 100)
        r = np.random.randint(timeCounter - 5 if timeCounter - 5 > 0 else 0 ,timeCounter + 2)
        timeCounter += int(p/3)
        
        fileData.append([p,r])
        
    with open(f'PTSZ-instancje/136723/in136723_{x}.txt', "w") as f:
        f.write(f"{str(fileData[0])}\n")
        
        m0 = np.random.randint(50,100)
        m1 = np.random.randint(50,100)
        m2 = np.random.randint(50,60)
        m3 = np.random.randint(50,100)
        m4 = np.random.randint(99,100)
        machines = np.array([m0,m1,m2,m3,m4])
        machines =  machines /max(machines)
        machines = np.random.choice( machines, 5, replace=False )
        f.write(f"{' '.join([str(x) for x in machines])}\n")
        
        instances = fileData[1:]
        random.shuffle(instances)
        for line in instances:            
            f.write(f"{' '.join([str(x) for x in line])}\n")

# Verify

In [5]:
x = 50

In [101]:
data

{1: {'p': 89, 'r': 588, 'c': 1123.578947368421},
 2: {'p': 78, 'r': 131, 'c': 519.4736842105262},
 3: {'p': 36, 'r': 405, 'c': 1369.5263157894738},
 4: {'p': 52, 'r': 363, 'c': 399.0},
 5: {'p': 64, 'r': 441, 'c': 419.0},
 6: {'p': 34, 'r': 8, 'c': 788.0},
 7: {'p': 60, 'r': 685, 'c': 87.35294117647061},
 8: {'p': 79, 'r': 299, 'c': 175.01470588235293},
 9: {'p': 38, 'r': 263, 'c': 772.0},
 10: {'p': 23, 'r': 210, 'c': 1155.0},
 11: {'p': 21, 'r': 638, 'c': 240.57352941176475},
 12: {'p': 24, 'r': 421, 'c': 563.9411764705883},
 13: {'p': 39, 'r': 245, 'c': 857.7368421052631},
 14: {'p': 4, 'r': 495, 'c': 5.012658227848078},
 15: {'p': 23, 'r': 3, 'c': 1339.0},
 16: {'p': 89, 'r': 496, 'c': 847.578947368421},
 17: {'p': 49, 'r': 374, 'c': 465.1052631578947},
 18: {'p': 5, 'r': 351, 'c': 933.0},
 19: {'p': 12, 'r': 550, 'c': 827.0},
 20: {'p': 11, 'r': 342, 'c': 16.014705882352928},
 21: {'p': 61, 'r': 667, 'c': 406.8088235294117},
 22: {'p': 31, 'r': 50, 'c': 364.0},
 23: {'p': 40, 'r':

In [68]:
taskQueue

{'m0': [28,
  22,
  42,
  43,
  4,
  6,
  5,
  35,
  24,
  23,
  47,
  37,
  50,
  44,
  18,
  27,
  15,
  10,
  19],
 'm1': [20, 8, 7, 30, 11, 29, 32, 12, 21, 26],
 'm2': [34],
 'm3': [45, 25, 36, 2, 41, 17, 33, 9, 13, 31, 16, 48, 40, 1, 3, 46, 49],
 'm4': [14, 39, 38]}

In [69]:
machines

{'m0': {'speed': '0.9583333333333334', 'readyTime': 0},
 'm1': {'speed': '0.5729166666666666', 'readyTime': 0},
 'm2': {'speed': '0.010416666666666666', 'readyTime': 0},
 'm3': {'speed': '1.0', 'readyTime': 0},
 'm4': {'speed': '0.20833333333333334', 'readyTime': 0}}

In [103]:
def calculateCost(data, taskQueue, machines):
    calculatedWeight = 0
    for key, tasks in taskQueue.items():
        print(f"m: {key}")
        timeCounter = 0
        for task in tasks:
            print(f"task: {task}, {data[task]}")
            while machines[key]['readyTime'] > timeCounter or data[task]['r'] > timeCounter:
                timeCounter += 1 
            print(f"timeCounter: {timeCounter}")
            print(f"machines[key]['readyTime']: {machines[key]['readyTime']}")
            
            machines[key]['readyTime'] = timeCounter + (data[task]['p'] / float(machines[key]['speed']) )
            
            print(f"machines[key]['readyTime']: {machines[key]['readyTime']}")
            
            data[task]['c'] = timeCounter + (data[task]['p'] / float(machines[key]['speed']) ) - data[task]['r']
            
            
            print(f"data[task]['c']: {data[task]['c']}")
            print(f"realy p: {(data[task]['p'] / float(machines[key]['speed']) )}")
            print()
        #    if timeCounter < data[task]['r']:
        #        timeCounter = data[task]['r']
            # add p time to current time
        #    timeCounter += data[task]['p']
      #      c = timeCounter
      #      data[task]['c'] = c
    
    for key, value in data.items():
        print(f"{key} -> {value}")
    # check correctness
    if not data[task]["r"] <= data[task]["c"] - data[task]["p"]: 
        print("Error")

    # verify cost
    if data[task]["c"] > data[task]["d"]:
        calculatedWeight += data[task]["w"]
    #print(calculatedWeight)
    return calculatedWeight

In [107]:
motherFolder = "out" # "PTSZ-instancje"
data = {}
indicies = ['136723'] #["136805", "136792", "136683", "132231", "136730", "136682", "136764", "136782", "136723", "136778", "136309", "136718", "136315", "136759"]
for index  in indicies:
    print(index)
    for samples in nSamples:
        taskQueue = {}
        with open(f'{motherFolder}/{index}/out{index}_{samples}.txt', "r") as result:
            res = result.read().split('\n')
            weight = int(res[0])
            for i, name in enumerate(machineNames):
                gen = (t for t in res[i+1].split(" ") if t.isdigit())
                taskQueue[name] = [int(t) for t in gen]
    
        with open(f'PTSZ-instancje/{index}/in{index}_{samples}.txt', "r") as instance:
            d = instance.read().split('\n')
            n = int(d[0])
            machines = {}
            for name, speed in zip(machineNames, d[1].split(' ')):
                machines[name] = {'speed':speed, 'readyTime':0}
            for i,line in enumerate(d[2:]):
                if line == '':
                    continue
                p,r = line.split(' ')
                p,r = int(p), int(r)
                data[i+1] = {"p": p, "r": r, "c": 0 }
        
        calculatedWeight = calculateCost(data, taskQueue, machines)
        print(calculatedWeight)
        #if calculatedWeight != weight:
        #    print(f"wrong weight. Calculated: {calculatedWeight}, read: {weight}")
        #else:
        #    print(f"Weight: {calculatedWeight} confirmed")
    break
    print()
    print()
        
        

136723
m: m0
task: 21, {'p': 31, 'r': 83, 'c': 0}
timeCounter: 83
machines[key]['readyTime']: 0
machines[key]['readyTime']: 143.1764705882353
data[task]['c']: 60.176470588235304
realy p: 60.1764705882353

task: 2, {'p': 67, 'r': 137, 'c': 0}
timeCounter: 144
machines[key]['readyTime']: 143.1764705882353
machines[key]['readyTime']: 274.05882352941177
data[task]['c']: 137.05882352941177
realy p: 130.05882352941177

task: 29, {'p': 65, 'r': 274, 'c': 0}
timeCounter: 275
machines[key]['readyTime']: 274.05882352941177
machines[key]['readyTime']: 401.1764705882353
data[task]['c']: 127.1764705882353
realy p: 126.1764705882353

task: 38, {'p': 37, 'r': 402, 'c': 0}
timeCounter: 402
machines[key]['readyTime']: 401.1764705882353
machines[key]['readyTime']: 473.8235294117647
data[task]['c']: 71.8235294117647
realy p: 71.82352941176471

task: 24, {'p': 33, 'r': 480, 'c': 0}
timeCounter: 480
machines[key]['readyTime']: 473.8235294117647
machines[key]['readyTime']: 544.0588235294117
data[task]['c']:

KeyError: 'd'

In [66]:
machines

{'m0': {'speed': '0.9583333333333334', 'readyTime': 0},
 'm1': {'speed': '0.5729166666666666', 'readyTime': 0},
 'm2': {'speed': '0.010416666666666666', 'readyTime': 0},
 'm3': {'speed': '1.0', 'readyTime': 0},
 'm4': {'speed': '0.20833333333333334', 'readyTime': 0}}

In [11]:
# p - persistance
# r - ready time
# d - oczekiwany termin zakończenia
# w - weight
# Cj oznacza moment zakończenia wykonywania zadania Jj w uszeregowaniu

# p r d w
# r  >= c - p

# TaskScheduler

In [106]:
a,b,c,d = (1,1,1,1)
totalCost = 0
data = {}

indicies = ['136723']#["136805", "136792", "136683", "132231", "136730", "136682", "136764", "136782", "136723", "136778", "136309", "136718", "136315", "136759"]
for index  in indicies:
    for samples in nSamples:
        with open(f'PTSZ-instancje/{index}/in{index}_{samples}.txt', "r") as instance:
            d = instance.read().split('\n')
            n = int(d[0])
            machines = {}
            for name, speed in zip(machineNames, d[1].split(' ')):
                machines[name] = {'speed':speed, 'readyTime':0, 'gain':0}
            for i,line in enumerate(d[2:]):
                if line == '':
                    continue
                p,r = line.split(' ')
                p,r = int(p), int(r)
                data[i+1] = {"p": p, "r": r, "c": 0, "penalty": 0 }
            dataCopy = copy.deepcopy(data)
        
        calculatedWeight = 0
        timeCounter = 0
        outSchedule = {}
        for name in machineNames:
            outSchedule[name] = []
        
        start = time.time()
        while len(data) > 0:
            readyToGo = {}
            for key, task in data.items():
                if task['r'] <= timeCounter: # add penalty ( (timeCounter+p-d)*w )
                    endTime = timeCounter+task['p']
                    penaltyEndTask = ( endTime if endTime > 0 else 0 ) * 1
                    penaltyPersistance = task['p']
                    task['penalty'] = a*penaltyEndTask + c*penaltyPersistance
                    readyToGo[key] = task
                
            if len(readyToGo) == 0: # currently there is no task to go
                #print("upsi readyToGo")
                timeCounter += 1
                continue
            #print(f"readyToGo: {readyToGo}")

            # select task
            readyToGoSorted = sorted(readyToGo.items(), key=lambda y: (y[1]['penalty']), reverse=True)
            selectedTask = readyToGoSorted[0]
            
            # select machine
            readyMachines = [d for d in zip(machines.keys(), machines.values() ) if d[1]['readyTime'] <= timeCounter]
            if len(readyMachines) == 0: # currently there is free machine
                #print("upsi readyMachines")
                timeCounter += 1
                continue
            
            
            for name, machine in machines.values():
                howLongItWillBeDisabled = 0 ##########################################
                speed = machines[name]['speed']
                
                machines[name]['gain'] = speed
            selectedMachine = max(readyMachines, key=lambda y: (y[1]['speed']))
            
            print(f"timeCounter: {timeCounter}")
            print(f"selectedTask: {selectedTask}")
            
            # remove task from list active tasks, add time, append outschedule
            data.pop(selectedTask[0])
            
            print(f"selectedMachine: {selectedMachine}")
            #timeCounter += 1
            machines[selectedMachine[0]]['readyTime'] = timeCounter + (selectedTask[1]['p'] / float(selectedMachine[1]['speed']) )
            print(machines)
            
            print()
            
            outSchedule[selectedMachine[0]].append(selectedTask[0])    
        
        end = time.time()
        #print(end-start)
        
        #cost = calculateCost(dataCopy, outSchedule)
        cost = 5
        #print(cost)
        totalCost += cost
        
        print(outSchedule)
        
        with open(f'out/{index}/out{index}_{samples}.txt', "w") as f:
            f.write(f"{cost}\n")         
                #f.write(f"{' '.join([str(x) for x in line])}\n")
            for name in machineNames:
                f.write(f"{' '.join([str(x) for x in outSchedule[name]])}\n")
        
        break
        
print(totalCost)

timeCounter: 0
selectedTask: (28, {'p': 89, 'r': 0, 'c': 0, 'penalty': 178})
selectedMachine: ('m4', {'speed': '1.0', 'readyTime': 0})
{'m0': {'speed': '0.5151515151515151', 'readyTime': 0}, 'm1': {'speed': '0.5252525252525253', 'readyTime': 0}, 'm2': {'speed': '0.7777777777777778', 'readyTime': 0}, 'm3': {'speed': '0.797979797979798', 'readyTime': 0}, 'm4': {'speed': '1.0', 'readyTime': 89.0}}

timeCounter: 29
selectedTask: (13, {'p': 71, 'r': 29, 'c': 0, 'penalty': 171})
selectedMachine: ('m3', {'speed': '0.797979797979798', 'readyTime': 0})
{'m0': {'speed': '0.5151515151515151', 'readyTime': 0}, 'm1': {'speed': '0.5252525252525253', 'readyTime': 0}, 'm2': {'speed': '0.7777777777777778', 'readyTime': 0}, 'm3': {'speed': '0.797979797979798', 'readyTime': 117.9746835443038}, 'm4': {'speed': '1.0', 'readyTime': 89.0}}

timeCounter: 49
selectedTask: (31, {'p': 50, 'r': 49, 'c': 0, 'penalty': 149})
selectedMachine: ('m2', {'speed': '0.7777777777777778', 'readyTime': 0})
{'m0': {'speed': '

In [33]:
x = {'m0':{'speed':2, 'readyTime':0},'m1':{'speed':3, 'readyTime':0},'m2':{'speed':1, 'readyTime':5}, }

In [18]:
zip(machines.keys(), machines.values() )

In [37]:
[d for d in x.values() if d['readyTime'] <= 0]

[{'speed': 2, 'readyTime': 0}, {'speed': 3, 'readyTime': 0}]

In [42]:
readyMachines = [d for d in x.values() if d['readyTime'] <= 0]
readyMachines = max(readyMachines, key=lambda y: (y['speed']))
readyMachines

{'speed': 3, 'readyTime': 0}

In [9]:
from scipy.optimize import minimize

In [10]:
x0 = (1, 1, 1, 1)

In [17]:
def costFunction(x):
    a,b,c,d = x
    totalCost = 0
    data = {}
    indicies = ["136805", "136792", "136683", "132231", "136730", "136682", "136764", "136782", "136723", "136778", "136309", "136718", "136315", "136759"]
    for index  in indicies:
        for samples in nSamples:
            with open(f'PTSZ-instancje/{index}/in{index}_{samples}.txt', "r") as instance:
                d = instance.read().split('\n')
                n = int(d[0])                
                for i,line in enumerate(d[1:]):
                    if line == '':
                        continue
                    p,r,d,w = line.split(' ')
                    p,r,d,w = int(p), int(r), int(d), int(w)
                    data[i+1] = {"p": p, "r": r, "d": d, "w": w, "c": 0, "penalty": 0 }
                dataCopy = copy.deepcopy(data)


            calculatedWeight = 0
            timeCounter = 0
            outSchedule = []

            start = time.time()
            while len(data) > 0:
                readyToGo = {}
                for key, task in data.items():
                    if task['r'] <= timeCounter: # add penalty ( (timeCounter+p-d)*w )
                        endTime = timeCounter+task['p']-task['d']
                        penaltyEndTask = ( endTime if endTime > 0 else 0 ) * task['w']
                        penaltyWeight = task['w']
                        penaltyPersistance = task['p']
                        penaltyDue = task['d'] - timeCounter
                        task['penalty'] = a*penaltyEndTask + b*penaltyWeight + c*penaltyPersistance - d*penaltyDue
                        readyToGo[key] = task

                if len(readyToGo) == 0: # currently there is no task to go
                    #print("upsi")
                    timeCounter += 1
                    continue
                #print(f"readyToGo: {readyToGo}")

                readyToGoSorted = sorted(readyToGo.items(), key=lambda y: (y[1]['penalty'], y[1]['w']), reverse=True)
                selectedTask = readyToGoSorted.pop(0)
                #print(timeCounter)
                #print(f"selectedTask: {selectedTask}")

                data.pop(selectedTask[0])
                timeCounter += selectedTask[1]['p']
                outSchedule.append(selectedTask[0])    

            end = time.time()
          #  print(int((end-start)*1000) if int((end-start)*1000) != 0 else 1)

            cost = calculateCost(dataCopy, outSchedule)
            #print(cost)
            totalCost += cost
    print(f"[{totalCost},{a},{b},{c},{d}],")
    return totalCost

In [18]:
options = {"maxiter":40, "disp":True}

In [23]:
res = minimize(costFunction, x0, method='TNC', options=options, tol=1e-2)

[499007,1.0,1.0,1.0,1183],
[499007,1.0,1.0,1.0,1183],
[499012,1.00000001,1.0,1.0,1183],
[499007,1.0,1.00000001,1.0,1183],
[499030,1.0,1.0,1.00000001,1183],
[499007,1.0,1.0,1.0,1183],
[499007,0.9999999936691033,1.0,0.999999970877875,1183],
[499007,0.9999999936691033,1.0,0.999999970877875,1183],
[499007,1.0000000036691032,1.0,0.999999970877875,1183],
[499007,0.9999999936691033,1.00000001,0.999999970877875,1183],
[499007,0.9999999936691033,1.0,0.999999980877875,1183],
[499007,0.9999999936691033,1.0,0.999999970877875,1183],
[499007,0.9999999936691033,1.0,0.999999970877875,1183],
[499007,0.9999999936691033,1.0,0.999999970877875,1183],
[499007,1.0000000036691032,1.0,0.999999970877875,1183],
[499007,0.9999999936691033,1.00000001,0.999999970877875,1183],
[499007,0.9999999936691033,1.0,0.999999980877875,1183],
[499007,0.9999999936691033,1.0,0.999999970877875,1183],
[499007,0.9999999968345517,1.0,0.9999999854389375,1183],
[499007,0.9999999968345517,1.0,0.9999999854389375,1183],
[499007,1.0000000

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x = [1,2,3]

In [ ]:
plt.plot(x)
plt.title("Minimalizacja funkcji celu")
plt.xaxis("iteracja")
plt.yaxis("Koszt")

In [94]:
costFunction(x)

1
7
13
25
17
19
26
33
38
61
1
4
10
17
29
42
64
85
99
140
1
4
11
19
30
45
63
152
111
133
1
3
15
16
25
32
48
61
82
88
1
1
6
13
30
36
48
64
94
108
1
3
6
11
19
27
38
52
62
79
1
3
8
22
29
38
56
71
105
124
1
4
10
17
26
39
53
69
105
107
1
1
1
2
2
4
7
6
9
10
1
3
6
10
16
26
33
54
57
68
1
3
7
15
22
45
45
56
75
108
1
1
1
1
2
4
5
7
8
11
1
4
11
21
32
47
74
87
110
146
1
3
5
8
14
28
34
39
54
84


385570

In [85]:
x = (-140036.8041378972, 2.5486165590751364, -0.1998934445512417, 1183)